In [ ]:
import chargeback_rpt.db_postgres_command as db_command

import chargeback_rpt.vm_data_utility as vm_util
import chargeback_rpt.vm_data_charger as vc
import chargeback_rpt.email_notifier as x_mail
import chargeback_rpt.file_directory_manager as fd_mn
import chargeback_rpt.vm_data_charger as charger

import pandas as pd
import numpy as np
import datetime  
import calendar
import os

from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
import sys 

In [ ]:
run_py=False

is_monthly=True

fix_mmyy=False

x_unit_str='GB'

table_vm="report_storeonce_info"
xcol_vm="catalyst_name"

storeonce_cost_key= 'storeonce'
storeonce_cost='Cost(฿)'

storeonce_Total='TotalSize'
storeonce_Usage='UsageSize'

jobType=13  # Import StoreOnce


all_cols=['id','catalyst_name','site_name','created_date','user_size','disk_size','price_thb_month_gb','cost','month','year']
pdfReport_cols=['site_name','disk_size','cost']
pdfReportBySite_cols=['site_name','user_size','disk_size','cost']

numberic_cols=['user_size','disk_size','cost','vat','total_cost']



overview_dict={}


In [ ]:


x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
year_x=x_datenow.strftime('%Y')

if fix_mmyy:
 month_param='09'
 year_param='2022'
else:
 month_param=month_x
 year_param=year_x

In [ ]:
if  run_py:

    press_y='n'
    ok=False
    
    if len(sys.argv) > 1:
        if sys.argv[1]=='0'  and sys.argv[2]=='0':
         month_param=month_x
         year_param=year_x 
         ok=True   
         is_only_check_data=bool(int(sys.argv[3]))
        else:
         raise Exception("Specify 0 0 as arguments only")   
    else:
        print("If you want current month and year.Press Enter ")
        month_param = int(input("Enter the month (1-12) : ") or month_x)
        year_param = int(input("Enter the year(such as 2021,2022,2023...) : ") or year_x)
        print(f"Do for month={month_param} and year={year_param}")
        press_y=input(f"Press y=True and n=False : ") 
        if press_y.lower()=='y':
         ok=True


    if ok==True:
        try:
         report_dt= datetime.datetime(int(year_param), int(month_param), 1)
         print(report_dt)   
         month_param=report_dt.strftime('%m')
         year_param=report_dt.strftime('%Y')   

    #      n = len(sys.argv) 
    #      print("Total arguments passed:", n) 
    #      for  param in  sys.argv:
    #        print(param)

        except Exception as ex:
         print(ex)   
         raise ex
    else:
         quit()

In [ ]:
folder_prefix_report='storeonce_chargeback'
excel_prefix_report='Storeonce_Detail'
pdf_overview_prefix='Overview_StoreOnce_Chargeback'

bill_info_sr=None

#def  storeonce_monthly_report(month_param,year_param,bill_info_sr=None,is_monthly=True):

# Start Transaction

In [ ]:
t_id=vm_util.creating_transaction(14,month_param,year_param)
print(f"Transaction ID: {t_id}")

list_error=[]
print(list_error)

# Move  to util
def check_error_point(tran_id,is_mail_module=True):
  print(list_error)
  if True in list_error:

    # if errror was given from sub-system, we will let sub-system send error eamail by themselves      
    if  is_mail_module==True:
     vm_util.collect_error_to_sent_mail(tran_id) 
    
    print("Building StoreOnce report occured error") 
    raise Exception("Program is teminated and check error from email and log_error.txt")
    
  list_error.clear()
  print(list_error)   

# Check proper data in order to enable buiding report

In [ ]:
print("Check proper data in order to enable the system to buid report")
def error_text(product, table,type_error):
    if  type_error==1:
     return f'{product} error due to Either no the complete job OR no row in {table} For  Month-{month_param} and Year-{year_param}.<br>'
    else:
     return f'{product} error due to no the complete job For  Month-{month_param} and Year-{year_param}.<br>'   
def check_valid_data_to_build_report():
    listErrorsBuidReport=[]
    
    job=charger.get_last_complete_job(month_param,year_param,jobType)
    print('StoreOnce Complete Job\n',job)
    if (job is not None) and (len(job)>0):
        vm_row=charger.check_existing_one_info('report_storeonce_info',month_param,year_param)
        print(f'Found at lest one row in StoreOnce For  Month-{month_param} and Year-{year_param}')
        listErrorsBuidReport.append(None)
    else:
        listErrorsBuidReport.append(error_text('StoreOnce','report_storeonce_info',1)); 
    return listErrorsBuidReport    
    
listErrorsBuidReport=check_valid_data_to_build_report()
list_errors= [ error for error in listErrorsBuidReport if  error is not None ] 
if len(list_errors)>0:
 for error in list_errors: 
    vm_util.add_error_to_database(24,error,t_id)

 vm_util.collect_error_to_sent_mail(t_id)    
 print("Building report occured error")  
 raise Exception((' ').join(list_errors))
 quit()
   


In [ ]:
# filename_detail=f'StoreOnce_Chargeback_Detail_{month_param}-{year_param}.xlsx'
# file_detail_report=os.path.join(report_path,filename_detail)

# load configuation data  by getting from database and Initialize  value  and set folder for building Report

In [ ]:
def clear_folder_after_complete(x_is_completed,x_is_monthly,x_zip_folder_path,x_report_path):
    
 fd_mn.delete_entire_directory(x_zip_folder_path)   
  
 if x_is_monthly == False:
  fd_mn.delete_entire_directory(x_report_path)
 else:
   if x_is_completed ==False    : 
     fd_mn.delete_entire_directory(x_report_path)

In [ ]:
try:
    print("Load configuation data  by getting from database and Initialize  value ")
    type_to_zip=vm_util.get_config_value('report_file_type',t_id).split(',')
    print(f"report file type: {type_to_zip}")

    temp_path=vm_util.get_config_value('temp_path',t_id)
    print(f"temp_path: {temp_path}")

    if is_monthly == True:
     report_rootpath = vm_util.get_config_value('report_storeonce_rootpath',t_id)
     report_folder=f'{month_param}-{year_param}'
     zip_filename= f'{folder_prefix_report}_{month_param}-{year_param}.zip'    
    else:
     report_rootpath =temp_path
     report_folder=f"storeonce_report_temp_{datetime.datetime.now().strftime('%d-%m-%y_%H%M%S')}"
     zip_filename= f"{folder_prefix_report}_{month_param}-{year_param}_{datetime.datetime.now().strftime('%d-%m-%y_%H%M%S')}.zip" 


    print(report_rootpath)   
    print(report_folder)
    print(zip_filename)
    
    
    overview_templateName=vm_util.get_config_value('storeonce_overview_template_path',t_id)
    site_templateName=vm_util.get_config_value('storeonce_site_template_path',t_id)

    overview_css_file=vm_util.get_config_value('css_file_path',t_id)
    css_file=vm_util.get_config_value('storeonce_css_file_path',t_id)

    img_file=vm_util.get_config_value('logo_png_path',t_id)
    overview_dict['Report_Logo']=img_file

    print(overview_templateName)
    print(site_templateName)
    print(overview_css_file)
    print(css_file)
    print(img_file)

    
except Exception as ex:
    list_error.append(True)
    print(str(ex))
    vm_util.add_error_to_database(3,str(ex),t_id)  

In [ ]:
try:

    print("Create report folder & file Name")
    report_path=os.path.join(report_rootpath,report_folder)
    print(report_path)
    fd_mn.create_directory(report_path)

    filename_detail=f'{excel_prefix_report}_{month_param}-{year_param}.xlsx'
    file_detail_report=os.path.join(report_path,filename_detail)
    print(file_detail_report)

    overview_pdf=os.path.join(report_path,f'{pdf_overview_prefix}_{month_param}-{year_param}.pdf')
    print(overview_pdf)

except Exception as ex:
    list_error.append(True)
    print(str(ex))
    vm_util.add_error_to_database(21,str(ex),t_id)

check_error_point(t_id)


# Calculate StoreOnce cost


In [ ]:
def master_item_cost(cost_key):
 try:
    # focusing on column_table_name,field_source_name
    sql_cmd="select * from master_item_cost where cost_key =(%s) "
    sql_params=(cost_key,)
    item=db_command.get_one_sql(db_command.get_postgres_conn(),sql_cmd,sql_params)
    if item is not None:
     sr_fieldInfo=pd.Series(item)
     return sr_fieldInfo
    else:
        return None
 except Exception as error:
     #raise  error
     return None




In [ ]:
def list_sites(siteList):
 try:

     sql="select * from site_info where site_name in  %(siteList_param)s "   
     paramx=  {"siteList_param":tuple(siteList)}
     valx=db_command.get_list_sql(db_command.get_postgres_conn(),sql,paramx)
     if(valx is not None):
      df=pd.DataFrame(valx)
      return df
     else:
      error_message = f'not found some site  in {siteList}'
      print(error_message)
      vm_util.add_error_to_database(25, error_message, t_id)
      raise Exception(error_message)
 except Exception as error:
      raise  error
      # return None

In [ ]:
print("Calculate StoreOnce cost")


storeonce_cost_item=master_item_cost(storeonce_cost_key)
cost_unit=storeonce_cost_item['cost_unit']

vax_cal_percent=float(vm_util.get_config_value('vax_cal_percent',transaction_id=t_id))

print(f"Cost Unit = {cost_unit} and percent vat ={vax_cal_percent} %")


dfx=vc.list_x_info(table_vm,xcol_vm,month_param,year_param)
dfx[f'price_thb_month_{x_unit_str.lower()}']=cost_unit
dfx['cost']=dfx['disk_size']*cost_unit
dfx['percent_vat']=vax_cal_percent

dfx['vat']=dfx['cost']*(dfx['percent_vat']/100)
dfx['vat']=dfx['vat'].round(2)

dfx['total_cost']=dfx['cost']+dfx['vat']

# dfx[numberic_cols]=dfx[numberic_cols].round(2)



siteList=dfx['catalyst_name'].unique()
dfSite=list_sites(siteList)
dfSite

In [ ]:
dfx=dfx.merge(dfSite[["site_name","site_full_name"]],how='left',left_on='catalyst_name',right_on='site_name')
dfx['site_name']=dfx['site_full_name']
dfx=dfx.drop(columns=['site_full_name','id','import_date'])

print(dfx.info())
dfx

# Create Detail Report of StoreOnce as Excel file

In [ ]:
try:
    print("Create Detail Report of StoreOnce as Excel file")
    writer=pd.ExcelWriter(file_detail_report,engine='xlsxwriter')
    dfx.to_excel(writer, sheet_name='storeonce_detail',index=False)
    writer.save()

except Exception as ex:
    list_error.append(True)
    print(str(ex))
    vm_util.add_error_to_database(24,str(ex),t_id)   

# Create Overview Report of StoreOnce

In [ ]:
print("Create Overview Report of StoreOnce as Pdf file")

In [ ]:
bill_info_sr=vm_util.get_initial_bill_info(int(month_param),int(year_param),t_id)
overview_dict['BillInfo']=bill_info_sr

overview_site=vm_util.get_config_value("main_cost_center",t_id)
cc_info_sr=vm_util.get_cost_center(overview_site,t_id)
overview_dict["CostCenterInfo"]=cc_info_sr

In [ ]:
df_pdf_overview=dfx[pdfReport_cols]
df_pdf_overview

In [ ]:
overview_dict['AllSites_Catalyst']=df_pdf_overview
overview_dict['Total_AllCost']=df_pdf_overview['cost'].sum()

In [ ]:
print("Print Overview Report")
overview_html=vm_util.get_content_template(overview_templateName,overview_dict, tran_id=t_id)
HTML(string=overview_html).write_pdf(overview_pdf,stylesheets=[overview_css_file])

# Create Each Site of Report as Pdf file

In [ ]:
print("Create Each Site of Report as Pdf file")

In [ ]:
def get_storeonce_constant_val_report():
 try:
     sql="select * from storeonce_constant_val_report"
     valx=db_command.get_list_sql(db_command.get_postgres_conn(),sql,None)
     if(valx is not None):
      df=pd.DataFrame(valx)
      return df
     else:
      error_message = f'not found data in from storeonce_constant_val_report'
      print(error_message)
      add_error_to_database(25, error_message, t_id)
      raise Exception(error_message)
 except Exception as error:
      raise  error
dfConstDataReport= get_storeonce_constant_val_report()
dfConstDataReport.set_index('key',inplace=True)
constDataReport_sr=dfConstDataReport['value']
constDataReport_sr

In [ ]:
def get_site_info(site_name,transaction_id):
 try:
#      site_name_upper = site_name.replace(' ', '').upper()
     site_name_upper = site_name.upper()
     sql="select * from site_info where upper(site_name)=(%s)"
     paramx=( site_name_upper,)
     valx=db_command.get_one_sql(db_command.get_postgres_conn(),sql,paramx)
     if(valx is not None):
      sr=pd.Series(valx)
      print(sr) 
      return sr
     else:
      error_message = f'not found:{site_name}'
      print(error_message)
      add_error_to_database(25, error_message, transaction_id)
      raise Exception(error_message)
 except Exception as error:
      raise  error
      # return None

In [ ]:
#pdfReportBySite_cols=['catalyst_name','user_size','disk_size','cost']
count_item=1
for index,item in dfx.iterrows():
    
    site_catalyst=item['catalyst_name']

    site_info_sr=(dfSite.query('site_name==@site_catalyst')).iloc[0,:]
#     print(site_info_sr)
    
    site_dict={'Report_Logo':img_file,'ConstDataReport':constDataReport_sr,
               'SiteInfo':site_info_sr , 'BillInfo':bill_info_sr,
               'ItemInfo':item
    }
    # Test price all value to print out as pdf
#     if count_item==1:
    #         for key,value in site_dict.items():   
    #             print(f"===================================={key}====================================")
    #             print(value)

    print(f"Print Report of {site_catalyst}")
    site_pdf=os.path.join(report_path,f'{site_catalyst}_{month_param}-{year_param}.pdf')
    print(site_pdf)

    site_html=vm_util.get_content_template(site_templateName,site_dict, tran_id=t_id)
#         print(site_html)
    
    HTML(string=site_html).write_pdf(site_pdf,stylesheets=[css_file])
    count_item+=1



# Create Zip Tile 

In [ ]:
try:
    print("Create Zip to acttach file on email")
    zip_folder_name=f"zip_report_{datetime.datetime.now().strftime('%d-%m-%y_%H%M%S')}"
    zip_folder_path = os.path.join(temp_path,zip_folder_name) 


    result_os=fd_mn.create_directory(zip_folder_path)

    zip_temp_file = os.path.join(zip_folder_path,zip_filename)  
    type_to_zip=['.pdf','.xlsx']

    print(zip_temp_file)   
    result_os=fd_mn.make_zip(report_path,type_to_zip,zip_temp_file)
    print(result_os)  

except Exception as ex:
     list_error.append(True)
     print(str(ex))   
     vm_util.add_error_to_database(24,str(ex),t_id)  
        

check_error_point(t_id)

# Email Notification

In [ ]:
print("Send mail")
#required_keys=['email_type','transaction_id','attached_file_path','content_data_dict']
try:
 content_data_dict= {
                 "ContentTitle":"List All StoreOnce Report in AttachedFile",
                }
 ok= x_mail.send_email(email_type='report_storeonce',transaction_id=t_id,
                 attached_file_path=zip_temp_file,content_data_dict=content_data_dict)

except Exception as ex:
 print(str(ex))   
 list_error.append(True)
 # delelte all folder
 clear_folder_after_complete(False,is_monthly,zip_folder_path,report_path)


check_error_point(t_id,False) 

# Delete temp folder report

In [ ]:
def clear_folder_after_complete(x_is_completed,x_is_monthly,x_zip_folder_path,x_report_path):
    
 fd_mn.delete_entire_directory(x_zip_folder_path)   
  
 if x_is_monthly == False:
  fd_mn.delete_entire_directory(x_report_path)
 else:
   if x_is_completed ==False    : 
     fd_mn.delete_entire_directory(x_report_path)
        
print("Delete temp folder and zip report")
clear_folder_after_complete(True,is_monthly,zip_folder_path,report_path)

# Completed Transaction

In [ ]:
try: 

    updated_rows=vm_util.created_transaction(t_id)
    print("completed building report")
    
except Exception as ex:
    list_error.append(True)
    print(ex)
    
check_error_point(t_id)